# Train and Deploy Custom Model in AWS

## Project: Train, Evaluate and Deploy Dog Identification App in SageMaker
---
### Why We're Here 
In this notebook, we will train and deploy a **custom model** in SageMaker. Specifically, the pretrained PyTorch model from  [Dog Breed Classifier](https://github.com/reedemus/dog_breed_classifier) project will be used as an example for this exercise. 
### The Road Ahead

We break the notebook into separate steps. Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Upload the dataset into an S3 bucket
* [Step 1](#step1): Create the custom model
* [Step 2](#step2): Completing a training script
* [Step 3](#step3): Training and deploying the custom model
* [Step 4](#step4): Evaluating the performance
---
<a id='step0'></a>
## Step 0: Upload the dataset to S3

We will import the AWS SageMaker libraries and define helper functions for handling the dataset. We will download the dog dataset from [this URL](https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip) and extract it before uploading them into the bucket.


In [ ]:
# import the required libraries
import requests
import boto3
import sagemaker
import zipFile as zf

Define `downloadFile` and `extractFile` helper functions to download the dataset.

In [ ]:
def downloadFile(file_url, file_name, dir=None, chunk_size=1024):
    '''
    Helper function to download file to specified directory

    :param file_url: file download URL
    :param file_name: file name to be saved.
    :param dir: path where file is saved other than current directory (Default = current working directory)
    :param chunk_size: size of file chunk to download (Default = 1024 bytes)
    :returns: None
    '''
    saved_file_path = file_name
    if dir != None and not os.path.exists(dir):
        os.mkdir(dir)
        saved_file_path = os.path.join(dir, file_name)

    r = requests.get(file_url, stream=True)
    total_size_in_bytes = len(r.content)
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True, desc=file_name)
    
    with open(saved_file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size):
            progress_bar.update(len(chunk))
            # writing one chunk at a time to file
            if chunk:
                f.write(chunk)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
       print("ERROR, something went wrong")
       return

def extractFile(file_name):
    '''
    Extracts compressed file in zip format into current directory
    
    :param file_name: file name
    :returns: None
    '''
    # create a zipfile object and extract it to current directory
    print("Extracting file...")
    with zf.ZipFile(file_name, 'r') as z:
        z.extractall()


Downlaod the dataset into current directory. The default folder after extraction is `dogImages/`.

In [ ]:
from glob import glob

dog_url = 'https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip'

downloadFile(dog_url, 'dogImages.zip')
extractFile('dogImages.zip')

# load filenames for human and dog images
dog_files = np.array(glob("dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total dog images.' % len(dog_files))

Create a sagemaker session and a default S3 bucket. Then upload to bucket.

In [ ]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

# Name of the dataset directory
data_dir = 'dogImages'

# set prefix, a descriptive name for a directory  
prefix = 'dog images'

# upload all data to S3
dataset = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(dataset)

<a id='step1'></a>
# Step 1: Create the custom model

Create a CNN model to classify dog breed using transfer learning.

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

# Using feature extraction approach
# =================================
# Freeze the weights for all of the network except the final fully connected(FC) layer.
# This last FC layer is replaced with a new one with random weights and only this layer is trained.
# https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#initialize-and-reshape-the-networks

# ResNet 152-layer model
model_transfer = models.resnet152(pretrained=True)

# Freeze the pre-trained weights,biases of all layers at first so it doesn't get updated during re-training
for param in model_transfer.parameters():
    param.requires_grad = False

# Get the number of input features in the last FC layer
# Reinitialize output features to number of dog breed classes
input_features = model_transfer.fc.in_features
DOG_BREEDS_NUM = 133
model_transfer.fc = nn.Linear(input_features, DOG_BREEDS_NUM)

print("ResNet-152 last fc layer:", models.resnet152().fc)
print("Our fc layer:", model_transfer.fc)

use_cuda = torch.cuda.is_available()
if use_cuda:
    model_transfer = model_transfer.cuda()

---
<a id='step2'></a>
## Step 2: Detect Dogs

In this section, we use a [pre-trained model](http://pytorch.org/docs/master/torchvision/models.html) to detect dogs in images.  

### Obtain Pre-trained VGG-16 Model

The code cell below downloads the VGG-16 model, along with weights that have been trained on [ImageNet](http://www.image-net.org/), a very large, very popular dataset used for image classification and other vision tasks.  ImageNet contains over 10 million URLs, each linking to an image containing an object from one of [1000 categories](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).  

---
<a id='step3'></a>
## Step 3: Create a CNN to Classify Dog Breeds (from Scratch)

Now that we have functions for detecting humans and dogs in images, we need a way to predict breed from images.  In this step, you will create a CNN that classifies dog breeds.  You must create your CNN _from scratch_ (so, you can't use transfer learning _yet_!), and you must attain a test accuracy of at least 10%.  In Step 4 of this notebook, you will have the opportunity to use transfer learning to create a CNN that attains greatly improved accuracy.

We mention that the task of assigning breed to dogs from images is considered exceptionally challenging.  To see why, consider that *even a human* would have trouble distinguishing between a Brittany and a Welsh Springer Spaniel.  

Brittany | Welsh Springer Spaniel
- | - 
<img src="https://github.com/reedemus/dog_breed_classifier/blob/transfer-learning/images/Brittany_02625.jpg?raw=1" width="100"> | <img src="https://github.com/reedemus/dog_breed_classifier/blob/transfer-learning/images/Welsh_springer_spaniel_08203.jpg?raw=1" width="200">

It is not difficult to find other dog breed pairs with minimal inter-class variation (for instance, Curly-Coated Retrievers and American Water Spaniels).  

Curly-Coated Retriever | American Water Spaniel
- | -
<img src="https://github.com/reedemus/dog_breed_classifier/blob/transfer-learning/images/Curly-coated_retriever_03896.jpg?raw=1" width="200"> | <img src="https://github.com/reedemus/dog_breed_classifier/blob/transfer-learning/images/American_water_spaniel_00648.jpg?raw=1" width="200">


Likewise, recall that labradors come in yellow, chocolate, and black.  Your vision-based algorithm will have to conquer this high intra-class variation to determine how to classify all of these different shades as the same breed.  

Yellow Labrador | Chocolate Labrador | Black Labrador
- | -
<img src="https://github.com/reedemus/dog_breed_classifier/blob/transfer-learning/images/Labrador_retriever_06457.jpg?raw=1" width="150"> | <img src="https://github.com/reedemus/dog_breed_classifier/blob/transfer-learning/images/Labrador_retriever_06455.jpg?raw=1" width="240"> | <img src="https://github.com/reedemus/dog_breed_classifier/blob/transfer-learning/images/Labrador_retriever_06449.jpg?raw=1" width="220">

We also mention that random chance presents an exceptionally low bar: setting aside the fact that the classes are slightly imabalanced, a random guess will provide a correct answer roughly 1 in 133 times, which corresponds to an accuracy of less than 1%.  

Remember that the practice is far ahead of the theory in deep learning.  Experiment with many different architectures, and trust your intuition.  And, of course, have fun!

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively).  You may find [this documentation on custom datasets](http://pytorch.org/docs/stable/torchvision/datasets.html) to be a useful resource.  If you are interested in augmenting your training and/or validation data, check out the wide variety of [transforms](http://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)!

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
root_dir = 'dogImages/'
train_dir = os.path.join(root_dir, 'train')
valid_dir = os.path.join(root_dir, 'valid')
test_dir = os.path.join(root_dir, 'test')
IMG_SIZE = 256

# Data augmentation to create a variety of test images so the model learn to generalize better.
# Output is a tensor.
preprocess_train = transforms.Compose([
                                    transforms.RandomResizedCrop(IMG_SIZE),
                                    transforms.RandomRotation(20),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225] )
                                        ])

# Data augmentation is not performed on validation and test datasets because the goal is not to create more data,
# but to resize and crop the images to the same size as the input image.
# Output is a tensor.
preprocess_valid_test = transforms.Compose([
                                    transforms.Resize(384),
                                    transforms.CenterCrop(IMG_SIZE),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225] )
                                        ])

train_dataset = datasets.ImageFolder(train_dir, transform=preprocess_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=preprocess_valid_test)
test_dataset = datasets.ImageFolder(test_dir, transform=preprocess_valid_test)

BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

loaders_scratch = { 'train':train_loader, 'valid':valid_loader, 'test':test_loader }
use_cuda = torch.cuda.is_available()

**Question 3:** Describe your chosen procedure for preprocessing the data. 
- How does your code resize the images (by cropping, stretching, etc)?  What size did you pick for the input tensor, and why?
- Did you decide to augment the dataset?  If so, how (through translations, flips, rotations, etc)?  If not, why not?


**Answer**:

Using Pytorch transforms API, I resize the images to 256 x 256 resolution, which is a squarish format typically used in computer vision tasks. Also, this size uses less RAM for training as compared to 1024 x 1024 or higher.

I decide to augment only the training dataset to create more variety of training images so the the model learns and generalizes better. This means the model can handle photos that are not shot perfectly in real life.

However, the images in validation and test datasets are not augmented heavily(except resizing and cropping) as the objective is to have actual images for evaluation and not to artificially improve validation accuracy, which will lead to overfitting.

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/stable/optim.html).  Save the chosen loss function as `criterion_scratch`, and the optimizer as `optimizer_scratch` below.

In [ ]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.Adam( model_scratch.parameters(), lr=0.001 )

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_scratch.pt'`.

In [ ]:
# the following import is required for training to be robust to truncated images
import numpy as np
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            # zero the gradients accumulated from the previous backward propagation steps,
            # make prediction, calculate the training loss, perform backpropagation, 
            # and finally update model weights and biases.
            optimizer.zero_grad()
            output = model(data)
            loss = criterion( output, target)
            loss.backward()
            optimizer.step()
            
            # record the average training loss, using something like
            # train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        
        ######################    
        # validate the model #
        ######################
        # switch model to evalution mode and disable gradient calculations to reduce memory usage
        # and speed up computations since no backpropagation is needed in evaluation.
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(loaders['valid']):
                # move to GPU
                if use_cuda:
                    data, target = data.cuda(), target.cuda()

                # get prediction from our model, calculate the validation loss
                output = model(data)
                loss = criterion( output, target)

                ## update the average validation loss
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))

            # print training/validation statistics 
            print('Epoch {}: \tTraining Loss: {:.3f} \tValidation Loss: {:.3f}'.format(
                epoch, train_loss, valid_loss))

            ## TODO: save the model if validation loss has decreased
            if valid_loss < valid_loss_min:
                print('Saving Model...')
                valid_loss_min = valid_loss
                torch.save(model.state_dict(),save_path)
    
    # return trained model
    return model

In [ ]:
# train the model
model_scratch = train(50, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, use_cuda, 'model_scratch.pt')

# save model to Google Drive
if isColabRunning:
    saveToDrive('model_scratch.pt')

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images.  Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 10%.

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (100. * correct / total, correct, total))

In [ ]:
# call test function    
test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

---
<a id='step4'></a>
## Step 4: Create a CNN to Classify Dog Breeds (using Transfer Learning)

You will now use transfer learning to create a CNN that can identify dog breed from images.  Your CNN must attain at least 60% accuracy on the test set.

### (IMPLEMENTATION) Specify Data Loaders for the Dog Dataset

Use the code cell below to write three separate [data loaders](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) for the training, validation, and test datasets of dog images (located at `dogImages/train`, `dogImages/valid`, and `dogImages/test`, respectively). 

If you like, **you are welcome to use the same data loaders from the previous step**, when you created a CNN from scratch.

>__Note:__ Previous loaders cannot be re-used because the input image was resized to **256 x 256**, but models trained on ImageNet are **224 x 224**. So we have to start over writing a new loader, *loaders_transfer*.


In [ ]:
## TODO: Specify data loaders
# Note: pretrained models from ImageNet requires input images to be shape (3 x h x w) with h,w >= 224 and 
#       normalized to mean,std dev as per ImageNet (mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225])
#
# Reference: https://pytorch.org/vision/stable/models.html
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Specify appropriate transforms, and batch_sizes
root_dir = 'dogImages/'
train_dir = os.path.join(root_dir, 'train')
valid_dir = os.path.join(root_dir, 'valid')
test_dir = os.path.join(root_dir, 'test')
# ResNet input image size
IMG_SIZE = 224
# mean and std deviation of models trained on Imagenet dataset
mean = [0.485, 0.456, 0.406]
std_dev = [0.229, 0.224, 0.225]

# Data augmentation to create a variety of test images so the model learn to generalize better.
# Output is a tensor.
preprocess_train = transforms.Compose([
                                    transforms.RandomResizedCrop(IMG_SIZE),
                                    transforms.RandomRotation(20),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize( mean, std_dev)
                                    ])

# Data augmentation is not performed on validation and test datasets because the goal is not to create more data,
# but to resize and crop the images to the same size as the input image.
# Output is a tensor.
preprocess_valid_test = transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.CenterCrop(IMG_SIZE),
                                    transforms.ToTensor(),
                                    transforms.Normalize( mean, std_dev)
                                    ])

train_dataset = datasets.ImageFolder(train_dir, transform=preprocess_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=preprocess_valid_test)
test_dataset = datasets.ImageFolder(test_dir, transform=preprocess_valid_test)

BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

loaders_transfer = { 'train':train_loader, 'valid':valid_loader, 'test':test_loader }

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify dog breed.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

# Using feature extraction approach
# =================================
# Freeze the weights for all of the network except the final fully connected(FC) layer.
# This last FC layer is replaced with a new one with random weights and only this layer is trained.
# https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#initialize-and-reshape-the-networks

# ResNet 152-layer model
model_transfer = models.resnet152(pretrained=True)

# Freeze the pre-trained weights,biases of all layers at first so it doesn't get updated during re-training
for param in model_transfer.parameters():
    param.requires_grad = False

# Get the number of input features in the last FC layer
# Reinitialize output features to number of dog breed classes
input_features = model_transfer.fc.in_features
DOG_BREEDS_NUM = 133
model_transfer.fc = nn.Linear(input_features, DOG_BREEDS_NUM)

print("ResNet-152 last fc layer:", models.resnet152().fc)
print("Our fc layer:", model_transfer.fc)

use_cuda = torch.cuda.is_available()
if use_cuda:
    model_transfer = model_transfer.cuda()

__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__
From [PyTorch website](https://pytorch.org/vision/stable/models.html), ResNet152 has the lowest Top-1 and Top-5 errors among all the ResNet architectures. Also, the model has very good accuracy with a deep 152 hidden layers. Since it is pre-trained on ImageNet, which contains dog breed classes, the model is a pretty good candidate for our use case.


### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [ ]:
# use same as before
import torch.optim as optim

criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.Adam( model_transfer.parameters(), lr=0.001 )

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [ ]:
# train the model
n_epochs = 40
model_transfer = train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# save model to Google Drive
if isColabRunning:
    saveToDrive('model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by your model.  

In [ ]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.
from PIL import Image
import torchvision.transforms as transforms
import torch
  
# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in loaders_transfer['train'].dataset.classes]

def predict_breed_transfer(img_path):
    # load the image and return the predicted breed
    image = Image.open(img_path).convert(mode='RGB')
    IMAGE_SIZE = 224
    # preprocess the image using transform
    prediction_transform = transforms.Compose([
                                        transforms.Resize(256),
                                        transforms.CenterCrop(IMAGE_SIZE),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                             std=[0.229, 0.224, 0.225] )
                                        ])
    image_tensor = prediction_transform(image).unsqueeze(0)
    # move to GPU
    if use_cuda:
        image_tensor = image_tensor.cuda()
    
    # set to evaluation mode for inferencing
    model_transfer.eval()
    idx = torch.argmax(model_transfer(image_tensor))
    return class_names[idx]

---
<a id='step5'></a>
## Step 5: Write your Algorithm

Write an algorithm that accepts a file path to an image and first determines whether the image contains a human, dog, or neither.  Then,
- if a __dog__ is detected in the image, return the predicted breed.
- if a __human__ is detected in the image, return the resembling dog breed.
- if __neither__ is detected in the image, provide output that indicates an error.

You are welcome to write your own functions for detecting humans and dogs in images, but feel free to use the `face_detector` and `dog_detector` functions developed above.  You are __required__ to use your CNN from Step 4 to predict dog breed.  

Some sample output for our algorithm is provided below, but feel free to design your own user experience!

![Sample Human Output](https://github.com/reedemus/dog_breed_classifier/blob/transfer-learning/images/sample_human_output.png?raw=1)


### (IMPLEMENTATION) Write your Algorithm

In [ ]:
### TODO: Write your algorithm.
### Feel free to use as many code cells as needed.
def run_app(img_path):
    '''handle cases for a human face, dog, and neither'''
    # open image
    img = Image.open(img_path)
    
    # predict its breed
    breed = predict_breed_transfer(img_path)
    
    if face_detector(img_path) == True:
        className = predict_breed_transfer(img_path)
        msg = "You are not a dog...but sure looks like a " + str(className)
    elif dog_detector(img_path) == True:
        className = predict_breed_transfer(img_path)
        msg = "I'm guessing your dog is a " + str(className) + "!"
    else:
        msg = "Not a dog or human...what are you?"

    plt.imshow(img)
    plt.axis('off')
    plt.title(msg)
    plt.show()

---
<a id='step6'></a>
## Step 6: Test Your Algorithm

In this section, you will take your new algorithm for a spin!  What kind of dog does the algorithm think that _you_ look like?  If you have a dog, does it predict your dog's breed accurately?  If you have a cat, does it mistakenly think that your cat is a dog?

### (IMPLEMENTATION) Test Your Algorithm on Sample Images!

Test your algorithm at least six images on your computer.  Feel free to use any images you like.  Use at least two human and two dog images.  

__Question 6:__ Is the output better than you expected :) ?  Or worse :( ?  Provide at least three possible points of improvement for your algorithm.

__Answer:__ (Three possible points for improvement)
<br> Yes, the outputs are more accurate than the model output from the CNN designed from scratch.
However, the model is unable to classify the last dog image which contains a yellow ball. Some improvements that can be tried are:
1. Add more variety of images to the dog train dataset, which includes a dog with a ball.
> a quick glance on the train dataset shows there is no dog images with a ball.
2. Increase the training sample size for this dog breed class by finding more photos.
3. Resize and crop the test image before feeding into the model.

In [ ]:
## TODO: Execute your algorithm from Step 6 on
## at least 6 images on your computer.
## Feel free to use as many code cells as needed.

## suggested code, below
for file in np.hstack((human_files[:3], dog_files[:3])):
    run_app(file)